# Testando a classe da MLP

In [1]:
%matplotlib widget
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns # a biblioteca 'seaborn' contém vários estilos para os gráficos do 'matpĺotlib'

# agora, melhoramos a qualidade de saida e de visualizacao da imagem 
# alem de mudar a fonte padrao para uma do latex
sns.set_style("ticks")
plt.rcParams['savefig.dpi'] = 200
plt.rcParams["figure.dpi"] = 150

plt.rcParams.update({
    "text.usetex": True,
    "font.family": "serif",
    "font.serif": ["Palatino"],
})
plt.style.use('dark_background')

### 1.2 Bibliotecas para MLP

In [2]:
import tensorflow as tf
physical_devices = tf.config.list_physical_devices('GPU')

from tensorflow import keras

### 1.3 Bibliotecas dos sistemas caóticos

In [3]:
import sys 
sys.path.insert(0, '../../../../scripts')

import timeseries as times
import mlp_model as mlp
import logisticmap as logmap

## 2. Gerando a série temporal do mapa logístico

Os valores para os parâmetros das equações serão os "básicos" apresentados no *paper* original. Utilizaremos  $r = 3.86$, o que faz com que o sistema esteja em caos.

In [4]:
x_inicial = 0.5
n_inicial = 0
n_iteracoes = 5000
r=3.86
estados = np.array([x_inicial, n_inicial])

In [5]:
mapa = logmap.MapaLogistico(estado_inicial=estados, r=r)

In [6]:
for n in range(0, n_iteracoes):
    mapa.iterar()
    estados = np.vstack((estados, mapa.ler_estado()))

In [7]:
x = estados[:, 0]
n = estados[:, 1]

In [8]:
fig, ax = plt.subplots()
ax.plot(n, x, color='Crimson')

ax.set_title("$100$ iterações iniciais da série temporal do Mapa Logístico\n para $r =$ " + str(r) + " com $x[0] =$ " + str(x[0]))
ax.set_ylabel('$x[n]$')
ax.set_xlabel('$n$')
ax.set_xlim(0,100)
    
ax.grid(True)
sns.despine()

plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

### 2.1 Dividindo em um conjunto de treinamento e de teste, para K = 4 e L = 3

In [9]:
K = 4
L = 3
tam_teste = 0.15
tam_val = 0.1

In [10]:
serie_temporal = times.SerieTemporal(x, K=K, L=L)

In [11]:
serie_temporal.criar_matrizes()

In [12]:
X_treino, X_val, X_teste, y_treino, y_val, y_teste = serie_temporal.dividir_treino_teste_validacao(tam_teste, tam_val)

## 3. Usando o modelo com a classe

In [13]:
batch_normalization = 'OFF'
activation = 'tanh'
init_mode = 'glorot_uniform'
n_neurons = 10
n_hidden_layers = 3

optimizer = 'Nadam'
learning_rate = 0.003
loss = "mean_squared_error"

batch_size = 2

In [14]:
MLP = mlp.ModeloMLP(input_size = K, name="MLP-logistic")

In [15]:
MLP.construir_mlp(batch_normalization = batch_normalization, activation = activation,
                  init_mode = init_mode, n_neurons = n_neurons,
                  n_hidden_layers = n_hidden_layers)

In [16]:
MLP.compilar(optimizer = optimizer, learning_rate = learning_rate, loss = loss)

In [17]:
MLP.visualizar()

Model: "MLP-logistic"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
camada_de_entrada (Dense)    (None, 4)                 20        
_________________________________________________________________
camada_intermediaria_1 (Dens (None, 10)                50        
_________________________________________________________________
camada_intermediaria_2 (Dens (None, 10)                110       
_________________________________________________________________
camada_intermediaria_3 (Dens (None, 10)                110       
_________________________________________________________________
camada_de_saida (Dense)      (None, 1)                 11        
Total params: 301
Trainable params: 301
Non-trainable params: 0
_________________________________________________________________


### Avaliando o modelo 5 vezes

In [18]:
mse_med, mse_dev = MLP.avaliar_modelo(X_treino, X_val, X_teste, y_treino, y_val, y_teste, n_repeticoes = 5, batch_size=2)

Média do erro quadrático médio: 0.0002566226198612724

Desvio padrão do erro quadrático médio: 4.0315061513693745e-05



### Retreinando uma última vez

In [19]:
MLP.construir_mlp(batch_normalization = batch_normalization, activation = activation,
                  init_mode = init_mode, n_neurons = n_neurons,
                  n_hidden_layers = n_hidden_layers)

In [20]:
MLP.compilar(optimizer = optimizer, learning_rate = learning_rate, loss = loss)

In [21]:
MLP.fit(X_treino, X_val, y_treino, y_val, batch_size=batch_size)

Epoch 1/100
1910/1910 [==============================] - 4s 2ms/step - loss: 0.0610 - val_loss: 0.0564
Epoch 2/100
1910/1910 [==============================] - 3s 2ms/step - loss: 0.0521 - val_loss: 0.0448
Epoch 3/100
1910/1910 [==============================] - 3s 2ms/step - loss: 0.0447 - val_loss: 0.0389
Epoch 4/100
1910/1910 [==============================] - 3s 2ms/step - loss: 0.0376 - val_loss: 0.0314
Epoch 5/100
1910/1910 [==============================] - 3s 1ms/step - loss: 0.0302 - val_loss: 0.0260
Epoch 6/100
1910/1910 [==============================] - 3s 1ms/step - loss: 0.0278 - val_loss: 0.0234
Epoch 7/100
1910/1910 [==============================] - 3s 1ms/step - loss: 0.0207 - val_loss: 0.0161
Epoch 8/100
1910/1910 [==============================] - 3s 2ms/step - loss: 0.0172 - val_loss: 0.0227
Epoch 9/100
1910/1910 [==============================] - 3s 2ms/step - loss: 0.0138 - val_loss: 0.0138
Epoch 10/100
1910/1910 [==============================] - 3s 1ms/step - l

In [22]:
y_pred = MLP.predict(X_teste)

In [23]:
fig, ax = plt.subplots()
ax.plot(n[len(n)-int(len(n)*tam_teste):,], y_teste, color='Crimson', label='Valor real')
ax.plot(n[len(n)-int(len(n)*tam_teste):,], y_pred, color='Silver', label='MLP')

ax.set_title("Comparação da predição da MLP com o valor real do mapa logístico\n utilizando a rede neural ótima no conjunto de teste")
ax.set_ylabel('$x[n]$')
ax.set_xlabel('$n$')
ax.set_xlim(4900, 5000)
    
ax.grid(True)
sns.despine()
ax.legend()

plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [24]:
MLP.salvar_modelo(nome_do_arquivo = "../../../../models/mlp-aprimorada/mlp-aprimorada-logistic")

INFO:tensorflow:Assets written to: ../../../../models/mlp-aprimorada/mlp-aprimorada-logistic/assets
O modelo foi salvo!
